In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
traindf = pd.read_csv('/kaggle/input/mkhackk/train.csv')
testdf = pd.read_csv('/kaggle/input/mkhackk/test.csv')

In [3]:
traindf.head()

In [4]:
traindf.info()

In [5]:
traindf.describe()

In [6]:
traindf.isna().sum()

In [7]:
traindf.duplicated().sum()

In [8]:
traindf.columns

In [9]:
traindf = traindf.drop(columns=['id','locality'],axis=1)

In [10]:
traindf

## Columnwise Analysis

Type

In [11]:
traindf.lease_type.value_counts()

In [12]:
traindf.type.value_counts()

In [13]:
traindf.furnishing.value_counts()

In [14]:
traindf.parking.value_counts()

In [15]:
traindf.facing.value_counts()

In [16]:
traindf.water_supply.value_counts()

In [17]:
traindf.building_type.value_counts()

In [18]:
df = pd.get_dummies(traindf,columns=['type','lease_type','furnishing','parking','facing','water_supply','building_type'], prefix=['type','lease','fur','parking','face','water','building'])

In [19]:
df.head()

In [20]:
import datetime
traindf['activation_date']

In [21]:
df['activation_date'] = pd.to_datetime(df['activation_date'])

In [22]:
df.head()

In [23]:
df.info()

In [24]:
df['amenities'][0]

In [25]:
m=df["amenities"]
m[0]

In [26]:
type(m[0])

In [27]:
type(m[0][0])

In [28]:
df1=df.copy()

In [29]:
for i in range(0,20499):
    str=df1.amenities[i]
    spl=str.split(',')
#     print(spl)
    for j in spl:
#         print(type(spl))
#         print(j)
#         print(type(j))
        spl2=j.split(":")
        if spl2[1]=="true":
            df1[spl2[0]]=1
        else:
            df1[spl2[0]]=0
        
        
df1                

In [30]:
columns=df1.columns
columns

In [31]:
df1.info()

In [32]:
df1=df1.drop(columns=["amenities"])


In [33]:
df1=df1.drop(columns=['{"LIFT"','"GYM"','{"AC"','"LIFT"','{"HK"','{"PARK"','{"INTERCOM"','{"SC"','{"SECURITY"'],axis=1)


In [34]:
df1.info()

In [35]:
import re
df1 = df1.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [36]:
from sklearn.model_selection import train_test_split
y= df1['rent']
x=df1.drop(columns=['rent','activation_date'],axis=1)

In [37]:
pd. set_option('display.max_rows', 20500) 

In [38]:
df1.corr()

In [39]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [40]:
x_train.shape

In [41]:
y_train.shape

In [42]:
x_test.shape

In [43]:
y_test.shape

In [44]:
from sklearn.linear_model import LinearRegression,PassiveAggressiveRegressor,TweedieRegressor,HuberRegressor,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor


In [45]:
from sklearn.preprocessing import StandardScaler, RobustScaler
sc = RobustScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [46]:
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)
dt.score(x_test,y_test)

In [47]:
kn = KNeighborsRegressor()
kn.fit(x_train,y_train)
kn.score(x_test,y_test)

In [48]:
rf = RandomForestRegressor(n_estimators = 500,max_depth=20)
rf.fit(x_train,y_train)
rf.score(x_test,y_test)

In [49]:
gb = GradientBoostingRegressor()
gb.fit(x_train,y_train)
gb.score(x_test,y_test)

In [50]:
xgb = XGBRegressor(n_estimators=500,learning_rate=0.2,max_depth=5)
xgb.fit(x_train,y_train)
xgb.score(x_test,y_test)

In [51]:
lb = lgb.LGBMRegressor(learning_rate=0.05,n_estimators=625)
lb.fit(x_train,y_train)
lb.score(x_test,y_test)

In [52]:
cat = CatBoostRegressor(verbose=0,learning_rate=0.08,iterations=1500)
cat.fit(x_train,y_train)
cat.score(x_test,y_test)

In [57]:
st = StackingRegressor([('xgb',XGBRegressor(n_estimators=500,learning_rate=0.2,max_depth=5)),('lgbr',lgb.LGBMRegressor(learning_rate=0.05,n_estimators=625)),('cbr',CatBoostRegressor(verbose = False,learning_rate=0.08,iterations=1500))],cv=5)
st.fit(x_train,y_train)
st.score(x_test,y_test)

In [58]:
estimator = [('stack',st),('lb',lb),('catb',cat)] 

st1 = StackingRegressor(estimators= estimator, cv=5)
st1.fit(x_train,y_train)
st1.score(x_test,y_test)

In [59]:
import pickle
pickle.dump(st, open('stmodel.pkl', 'wb'))